#  Pandas - Aplicação de Funções em Dataframes e Séries (pipe, apply, transform, applymap e map)

# Função de Aplicação
Para aplicar as suas próprias funções ou as de outra biblioteca aos objetos pandas, exitem três formas:
- Aplicação de função tablewise :pipe()
- Aplicação de função em linha ou coluna :apply()
- API de agregação : agg() e transform()
- Aplicando funções Elementwise :applymap()

### Aplicação da função tablewise 
DataFrames e Seriespode ser passado para funções. 
No entanto, se a função precisar ser chamada de forma encadeada, considere o uso do método pipe().


In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('totaisestadoscomregiao_arr.csv', sep=';',names =['REGIAO','UF','CASOS','OBITOS'])
df.head()

,REGIAO,UF,CASOS,OBITOS
0,Centro-Oeste,DF,955540,12025
1,Centro-Oeste,GO,2060518,28669
2,Centro-Oeste,MS,637169,11305
3,Centro-Oeste,MT,921447,15253
4,Nordeste,AL,348701,7355


In [2]:
df['OBITOSMILHARES'] = df['OBITOS']/1000
df['CASOSMILHARES'] = df['CASOS']/1000
df['LETALIDADE'] = df['OBITOS']/ df['CASOS'] * 100
df.head()


,REGIAO,UF,CASOS,OBITOS,OBITOSMILHARES,CASOSMILHARES,LETALIDADE
0,Centro-Oeste,DF,955540,12025,12.025,955.540,1.258451
1,Centro-Oeste,GO,2060518,28669,28.669,2060.518,1.391349
2,Centro-Oeste,MS,637169,11305,11.305,637.169,1.774255
3,Centro-Oeste,MT,921447,15253,15.253,921.447,1.655331
4,Nordeste,AL,348701,7355,7.355,348.701,2.109257


In [3]:
def calculaobitosemmilhares(df):
    df['OBITOSMILHARES'] = df['OBITOS']/1000
    return df

def calculacasosemmilhares(df):
    df['CASOSMILHARES'] = df['CASOS']/1000
    return df

def calculaletalidades(df):
    df['LETALIDADE'] = df['OBITOS']/ df['CASOS'] * 100
    return df

In [4]:
df = df[['REGIAO','UF','CASOS','OBITOS']]
df.head()

,REGIAO,UF,CASOS,OBITOS
0,Centro-Oeste,DF,955540,12025
1,Centro-Oeste,GO,2060518,28669
2,Centro-Oeste,MS,637169,11305
3,Centro-Oeste,MT,921447,15253
4,Nordeste,AL,348701,7355


In [5]:
calculaletalidades(calculaobitosemmilhares(calculacasosemmilhares(df))).head()

,REGIAO,UF,CASOS,OBITOS,CASOSMILHARES,OBITOSMILHARES,LETALIDADE
0,Centro-Oeste,DF,955540,12025,955.540,12.025,1.258451
1,Centro-Oeste,GO,2060518,28669,2060.518,28.669,1.391349
2,Centro-Oeste,MS,637169,11305,637.169,11.305,1.774255
3,Centro-Oeste,MT,921447,15253,921.447,15.253,1.655331
4,Nordeste,AL,348701,7355,348.701,7.355,2.109257


### É equivalente a:

In [6]:
df = pd.read_csv('totaisestadoscomregiao_arr.csv', sep=';',names =['REGIAO','UF','CASOS','OBITOS'])
df.head()

,REGIAO,UF,CASOS,OBITOS
0,Centro-Oeste,DF,955540,12025
1,Centro-Oeste,GO,2060518,28669
2,Centro-Oeste,MS,637169,11305
3,Centro-Oeste,MT,921447,15253
4,Nordeste,AL,348701,7355


In [7]:
(df.pipe(calculaobitosemmilhares)
    .pipe(calculacasosemmilhares)
    .pipe(calculaletalidades)
)
df.head()

,REGIAO,UF,CASOS,OBITOS,OBITOSMILHARES,CASOSMILHARES,LETALIDADE
0,Centro-Oeste,DF,955540,12025,12.025,955.540,1.258451
1,Centro-Oeste,GO,2060518,28669,28.669,2060.518,1.391349
2,Centro-Oeste,MS,637169,11305,11.305,637.169,1.774255
3,Centro-Oeste,MT,921447,15253,15.253,921.447,1.655331
4,Nordeste,AL,348701,7355,7.355,348.701,2.109257


### O Pandas recomenda o uso do pipe pois torna mais fácil usar suas próprias funções ou as funções de outra biblioteca encadeadas com os métodos do pandas.


In [8]:
(df.groupby('REGIAO')
    .pipe(lambda x: x.sum())
    .pipe(calculaobitosemmilhares)
    .pipe(calculacasosemmilhares)
    .pipe(lambda x: x.cumsum())
    .head()
)

,UF,CASOS,OBITOS,OBITOSMILHARES,CASOSMILHARES,LETALIDADE
REGIAO,,,,,,
Centro-Oeste,DFGOMSMT,4574674,67252,67.252,4574.674,6.079386
Nordeste,DFGOMSMTALBACEMAPBPEPIRNSE,12146631,204209,204.209,12146.631,22.623923
Norte,DFGOMSMTALBACEMAPBPEPIRNSEACAMAPPARORRTO,15130672,256323,256.323,15130.672,33.156882
Sudeste,DFGOMSMTALBACEMAPBPEPIRNSEACAMAPPARORRTOESMGRJSP,30725146,600860,600.860,30725.146,41.102084
Sul,DFGOMSMTALBACEMAPBPEPIRNSEACAMAPPARORRTOESMGRJ...,38991809,714078,714.078,38991.809,45.129409


### Aplicação de função em linha ou coluna 

As funções arbitrárias podem ser aplicadas ao longo dos eixos de um DataFrame usando o método apply(), que, como os métodos de estatística descritiva, tem um axis argumento opcional

In [37]:
df = pd.read_csv('totaisestadoscomregiao_arr.csv', sep=';'
                 ,names =['REGIAO','UF','CASOS','OBITOS'])
df.head()

,REGIAO,UF,CASOS,OBITOS
0,Centro-Oeste,DF,955540,12025
1,Centro-Oeste,GO,2060518,28669
2,Centro-Oeste,MS,637169,11305
3,Centro-Oeste,MT,921447,15253
4,Nordeste,AL,348701,7355


In [38]:
df[['CASOS', 'OBITOS']].apply(np.mean)

CASOS     1.444141e+06
OBITOS    2.644733e+04
dtype: float64

In [39]:
dfT = df[['CASOS', 'OBITOS']].T
dfT

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
CASOS,955540,2060518,637169,921447,348701,1844481,1508135,501121,725917,1235360,...,505111,190318,382320,1384800,4335714,2966219,6907741,3031840,3144956,2089867
OBITOS,12025,28669,11305,15253,7355,32034,28215,11103,10669,23240,...,7527,2202,4305,15214,66850,78238,184235,47029,43044,23145


In [40]:
dfT.apply(np.mean, axis=1)

CASOS     1.444141e+06
OBITOS    2.644733e+04
dtype: float64

In [41]:
df[['CASOS', 'OBITOS']].apply(lambda x: x.max() - x.min())

CASOS     6738116
OBITOS     182152
dtype: int64

In [42]:
df[['CASOS', 'OBITOS']].apply(np.cumsum).tail()

,CASOS,OBITOS
22,23817405,416625
23,30725146,600860
24,33756986,647889
25,36901942,690933
26,38991809,714078


In [43]:
df[['CASOS', 'OBITOS']].apply(np.log).tail()

,CASOS,OBITOS
22,14.902799,11.267511
23,15.748153,12.123967
24,14.924680,10.758520
25,14.961310,10.669978
26,14.552611,10.049534


O método apply() também pode ser passado um nome de método como texto

In [44]:
df[['CASOS', 'OBITOS']].apply('mean')

CASOS     1.444141e+06
OBITOS    2.644733e+04
dtype: float64

In [45]:
df[['CASOS', 'OBITOS']].T.apply('mean', axis=1)

CASOS     1.444141e+06
OBITOS    2.644733e+04
dtype: float64

In [46]:
df[['CASOS', 'OBITOS']].apply(lambda x: x.idxmax())

CASOS     23
OBITOS    23
dtype: int64

### Pode passar argumentos adicionais e argumentos de palavra-chave para o método apply() .

In [47]:
def valores_em_outra_unidade(x, divisor=1000):
    return (x / divisor)
    

In [48]:
df[['CASOS', 'OBITOS']].apply(valores_em_outra_unidade, divisor=1000).head()

,CASOS,OBITOS
0,955.540,12.025
1,2060.518,28.669
2,637.169,11.305
3,921.447,15.253
4,348.701,7.355


# API Transform 
- O método transform() retorna um objeto indexado com o mesmo (mesmo tamanho) do original.
- Essa api é muito semelhante à API .agg.

In [49]:
df = pd.read_csv('totaisregioes_arr.csv', sep=';',names =['REGIAO','CASOS','OBITOS'])
df.head()

,REGIAO,CASOS,OBITOS
0,Centro-Oeste,4574674,67252
1,Nordeste,7571957,136957
2,Norte,2984041,52114
3,Sudeste,15594474,344537
4,Sul,8266663,113218


In [24]:
df[3:5] = df[3:5].assign(CASOS = lambda x: x['CASOS'] * -1, OBITOS = lambda x: x['OBITOS'] * -1)
df.head()

,REGIAO,CASOS,OBITOS
0,Centro-Oeste,4574674,67252
1,Nordeste,7571957,136957
2,Norte,2984041,52114
3,Sudeste,-15594474,-344537
4,Sul,-8266663,-113218


### Transforme todo o quadro. 
.transform() permite funções de entrada como: uma função NumPy, um nome de função de string ou uma função definida pelo usuário.

In [25]:
df[['CASOS', 'OBITOS']].transform(np.abs)

,CASOS,OBITOS
0,4574674,67252
1,7571957,136957
2,2984041,52114
3,15594474,344537
4,8266663,113218


In [26]:
df[['CASOS', 'OBITOS']].transform('abs')


,CASOS,OBITOS
0,4574674,67252
1,7571957,136957
2,2984041,52114
3,15594474,344537
4,8266663,113218


In [27]:
df[['CASOS', 'OBITOS']].transform(lambda x: x.abs())

,CASOS,OBITOS
0,4574674,67252
1,7571957,136957
2,2984041,52114
3,15594474,344537
4,8266663,113218


### Passar uma única função para .transform()  com um Series  resultará em uma única Series em retorno.

In [28]:
df['CASOS'].transform(np.square)

0     20927642206276
1     57334532809849
2      8904500689681
3    243187619336676
4     68337717155569
Name: CASOS, dtype: int64

### Transforme com várias funções 
A passagem de várias funções produzirá uma coluna MultiIndexed DataFrame. O primeiro nível será os nomes das colunas do quadro original; o segundo nível serão os nomes das funções de transformação.

In [51]:
df.dtypes

REGIAO    object
CASOS      int64
OBITOS     int64
dtype: object

In [52]:
df[['CASOS', 'OBITOS']].transform([np.abs, lambda x: x /1000, np.log])

CASOS                         OBITOS                    
   absolute   <lambda>        log absolute <lambda>        log
0   4574674   4574.674  15.336046    67252   67.252  11.116202
1   7571957   7571.957  15.839962   136957  136.957  11.827422
2   2984041   2984.041  14.908789    52114   52.114  10.861189
3  15594474  15594.474  16.562427   344537  344.537  12.749957
4   8266663   8266.663  15.927741   113218  113.218  11.637070

Passar várias funções para uma série produzirá um DataFrame. Os nomes das colunas resultantes serão as funções de transformação.

In [53]:
df['CASOS'].transform([np.abs, lambda x: x /1000, np.log])

,absolute,<lambda>,log
0,4574674,4574.674,15.336046
1,7571957,7571.957,15.839962
2,2984041,2984.041,14.908789
3,15594474,15594.474,16.562427
4,8266663,8266.663,15.927741


### Transformando com um dicionario 
A passagem de um dicionário de funções permitirá a transformação seletiva por coluna.

In [54]:
df.transform({'CASOS': np.abs, 'OBITOS': lambda x: x /1000})

,CASOS,OBITOS
0,4574674,67.252
1,7571957,136.957
2,2984041,52.114
3,15594474,344.537
4,8266663,113.218


Passar um dicionário de listas irá gerar um MultiIndexed DataFrame com essas transformações seletivas.

In [55]:
df.transform({'CASOS': np.abs, 'OBITOS': [lambda x: x /1000, np.log, np.log10]})

CASOS   OBITOS                     
   absolute <lambda>        log     log10
0   4574674   67.252  11.116202  4.827705
1   7571957  136.957  11.827422  5.136584
2   2984041   52.114  10.861189  4.716954
3  15594474  344.537  12.749957  5.537236
4   8266663  113.218  11.637070  5.053915

In [56]:
df2 = df.transform({'CASOS': np.abs, 'OBITOS': [lambda x: x /1000, np.log, np.log10]})
df2

CASOS   OBITOS                     
   absolute <lambda>        log     log10
0   4574674   67.252  11.116202  4.827705
1   7571957  136.957  11.827422  5.136584
2   2984041   52.114  10.861189  4.716954
3  15594474  344.537  12.749957  5.537236
4   8266663  113.218  11.637070  5.053915

In [57]:
df2.columns = ['CASOS','OBITOS em mil','OBITOS log','OBITOS log10']
df2

,CASOS,OBITOS em mil,OBITOS log,OBITOS log10
0,4574674,67.252,11.116202,4.827705
1,7571957,136.957,11.827422,5.136584
2,2984041,52.114,10.861189,4.716954
3,15594474,344.537,12.749957,5.537236
4,8266663,113.218,11.637070,5.053915


## Aplicando funções elementwise 
Como nem todas as funções podem ser vetorizadas (aceitar matrizes NumPy e retornar outra matriz ou valor), os métodos applymap() em DataFrame aceitam qualquer função Python assumindo um único valor e retornando um único valor.

In [58]:
df = pd.read_csv('totaisregioes_arr.csv', sep=';',names =['REGIAO','CASOS','OBITOS'])
df.head()

,REGIAO,CASOS,OBITOS
0,Centro-Oeste,4574674,67252
1,Nordeste,7571957,136957
2,Norte,2984041,52114
3,Sudeste,15594474,344537
4,Sul,8266663,113218


In [59]:
def extrai_sigla(x):
    return ''.join([w[0] for w in x.split('-') ]) + ('E' if 'deste' in x else '')

In [60]:
df['REGIAO'].map(extrai_sigla)

0    CO
1    NE
2     N
3    SE
4     S
Name: REGIAO, dtype: object

In [61]:
def unidade_milhares(x):
    valor = x
    
    if type(valor) is int:
        valor = round(valor/1000,2)
    return valor
        
unidade_milhares(121323)      

121.32

In [62]:
def uppertext(x):
    return str(x).upper()

In [63]:
df

,REGIAO,CASOS,OBITOS
0,Centro-Oeste,4574674,67252
1,Nordeste,7571957,136957
2,Norte,2984041,52114
3,Sudeste,15594474,344537
4,Sul,8266663,113218


In [66]:
df.map(uppertext)

,REGIAO,CASOS,OBITOS
0,CENTRO-OESTE,4574674,67252
1,NORDESTE,7571957,136957
2,NORTE,2984041,52114
3,SUDESTE,15594474,344537
4,SUL,8266663,113218


In [67]:
df.map(unidade_milhares)

,REGIAO,CASOS,OBITOS
0,Centro-Oeste,4574.67,67.25
1,Nordeste,7571.96,136.96
2,Norte,2984.04,52.11
3,Sudeste,15594.47,344.54
4,Sul,8266.66,113.22


### Series.map() tem um recurso adicional; pode ser usado para “vincular” ou “mapear” facilmente valores definidos por uma série secundária. 

In [68]:
regioes = pd.Series({'Centro-Oeste':'CO', 'Nordeste':'NE', 'Norte' : 'N', 'Sudeste': 'SE', 'Sul': 'S'})
regioes

Centro-Oeste    CO
Nordeste        NE
Norte            N
Sudeste         SE
Sul              S
dtype: object

In [69]:
df['SIGLA_REGIAO'] = df['REGIAO'].map(regioes)
df

,REGIAO,CASOS,OBITOS,SIGLA_REGIAO
0,Centro-Oeste,4574674,67252,CO
1,Nordeste,7571957,136957,NE
2,Norte,2984041,52114,N
3,Sudeste,15594474,344537,SE
4,Sul,8266663,113218,S
